# Snowflake Setup
It's time to set up some access in Snowflake. Here's what you'll need:
- a user account for this code to use to issue iceberg table create commands
- that user account needs to have a default role assigned with access to:
  - create tables in your desired database/schema 
  - use a data warehouse
  - use the catalog and external volume you'll create below

That's it! Probably!

Here's a sample Snowflake script to get this cooking. The emojis point out places where you need to add values yourself:
```sql
// -----------------------------------------------
// create security admin assets
// -----------------------------------------------
use role securityadmin;
create role pyiceberg_mirroring_service_role;
grant role pyiceberg_mirroring_service_role to role sysadmin; 

// you can create these if they do not exist. This guide is just a starting point
grant usage on database my_target_database🪐 to role pyiceberg_mirroring_service_role;
grant ownership on schema my_target_database🪐.my_mirroring_schema🧊 to role pyiceberg_mirroring_service_role copy grants;
grant usage on warehouse some_cool_warehouse 🪐 to role pyiceberg_mirroring_service_role; // an xs is plenty here. Just doing some metadata ops 💪

create user pyiceberg_mirroring_service_user 
  password = '😎',
  must_change_password = false
  default_role = pyiceberg_mirroring_service_role,
  default_warehouse = some_cool_warehouse 🪐; 
  
grant role pyiceberg_mirroring_service_role to user pyiceberg_mirroring_service_user;
// -----------------------------------------------


// -----------------------------------------------
// create account admin assets 
// -----------------------------------------------
use role accountadmin;
create external volume databricks_unity_catalog_volume // I like to name these same as my warehouse, so enterprise_data_warehouse_volume for example 
  allow_writes=false
  storage_locations = ((
      name = 🌞'databricks_unity_catalog_volume'
      storage_provider = 'S3'
      storage_aws_role_arn = 🌞'arn:aws:iam::account_id:role/snowflake-databricks-my-cool-warehouse-volume' // call this what you like. We will create the role in your aws account after this
      storage_base_url = 🌞's3://my-s3-bucket-where-unity-catalog-puts-my-data/' // this is the s3 location of your tabular warehouse.
  ));

create catalog integration databricks_unity_catalog_my_cool_catalog
  catalog_source = object_store
  table_format = iceberg
  enabled = true
  comment = 'Catalog Integration for reading Databricks Unity Catalog Iceberg tables';
  
// let the service role use these nifty new iceberg objects
grant usage on integration databricks_unity_catalog_my_cool_catalog to role pyiceberg_mirroring_service_role;
grant usage on volume databricks_unity_catalog_volume to role pyiceberg_mirroring_service_role;
// -----------------------------------------------


// -----------------------------------------------
// Get details from newly-created integration 
// objects. 
// -----------------------------------------------
use role pyiceberg_mirroring_service_role;
use warehouse some_cool_warehouse 🪐;
describe external volume databricks_unity_catalog_volume;


// this query will give you 2 values you will need to create the snowflake access role in AWS
select 
  parse_json("property_value"::string):"STORAGE_AWS_ROLE_ARN"::string as storage_aws_iam_role_arn,
  parse_json("property_value"::string):"STORAGE_AWS_IAM_USER_ARN"::string as storage_aws_iam_user_arn,
  parse_json("property_value"::string):"STORAGE_AWS_EXTERNAL_ID"::string as storage_aws_external_id
  
from table(result_scan(last_query_id()))

where "parent_property"='STORAGE_LOCATIONS' and "property"='STORAGE_LOCATION_1';

```

🚧 Quick check!
- make sure to grab those three important fields from Snowflake -- or just keep them handy for the AWS step
- I attached a screenshot of my results. You should see different values but the same kinda thing after running the final SQL statement in the script above.

![image.png](attachment:image.png)

## 3. AWS Setup
No one panic, but it's time to log in to AWS and go to IAM and create a new policy + role 💪

### Create Policy
- let's start with the policy. We need to build a read only policy for your S3 location that holds the Iceberg warehouse you want to mirror

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion"
            ],
            "Resource": [
                "arn:aws:s3:::your_s3_bucket🦆/*"
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                "arn:aws:s3:::your_s3_bucket🦆"
            ],
            "Effect": "Allow"
        }
    ]
}
```

### Create Role, *with Trust Policy*
- AWS > IAM > Create Role
- Important! You MUST name this role identically to what you told Snowflake to look for. So make sure those values are the same. This will be the `STORAGE_AWS_IAM_ROLE_ARN` value you received in your final Snowflake SQL query in the previous Snowflake Setup step 💪
- we need to attach the IAM Policy that you created above to this role
- now for the tricky part -- let's add a trust relationship so snowflake can use this role
  - remember the Snowflake values from the snowflake step? We need those. You should have
      - Snowflake AWS User ARN
      - Snowflake AWS External ID
  - these are super important, make sure to copy them EXACTLY! I mean it!

Lastly, here is a sample of the trust policy you'll need.

```json
{
    "Version": "2008-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "snowflake user arn here ❄️"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "sts:ExternalId": "🏂 snowflake external ID here"
                }
            }
        }
    ]
}
```

💾 Suplex that save button and let's get down to business!


## 4. Pyiceberg Mirroring (aka, getting down to business 🕴️)
- remember everything we've been through. We're bonded after all this effort above
- keep the following values handy. I recommend putting a few of them in your `.env` file
  - you'll need your snowflake account id. You could stumble through the docs for this, OR just copy the results of running this in snowflake. Save this in `.env` as `SNOWFLAKE_ACCOUNT_IDENTIFIER`.
  ```sql
  select current_organization_name() || '-' || current_account_name() as snowflake_account_identifier;
  ```
  - you'll need your snowflake service user login name and password.
    - set these in your .env as `SNOWFLAKE_USERNAME` and `SNOWFLAKE_PASSWORD`
  - you tabular credential should already be tucked safe and sound in your .env file as `TABULAR_CREDENTIAL`

Now we're ready to cook 🍳

### ⚠️ Double check:
- Seriously, make sure you save that .env file. 
- if this is scary, you can ignore the `.env` file and just paste your credential in plaintext directly in this notebook -- but you should feel bad about your craftsmanship.


*One last note* -- you definitely don't have the same data I do. Make sure you use your own configs as required, but this should be a good starting point for you.

In [0]:
%pip install pyiceberg[pyarrow]
%pip install --upgrade pyparsing
%restart_python

In [0]:
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import TableAlreadyExistsError

# Databricks config
UC_CATALOG_TO_MIRROR = 'analytics_prod' # replace with the catalog name you want to mirror.
UC_CREDENTIAL  = dbutils.secrets.get(scope="randy_pitcher_workspace", key="databricks_pat")
UC_DATABRICKS_URL = f'{dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()}'
UC_CATALOG_URI = f'https://{UC_DATABRICKS_URL}/api/2.1/unity-catalog/iceberg'

# Snowflake config
SNOWFLAKE_VOLUME  = 'databricks_uc_volume' # You created this in your Snowflake script. Please copy/paste that value here
SNOWFLAKE_CATALOG = 'databricks_uc_catalog' # You created this in your Snowflake script. Please copy/paste that value here
SNOWFLAKE_USERNAME           = dbutils.secrets.get(scope="randy_pitcher_workspace", key="SNOWFLAKE_USERNAME")
SNOWFLAKE_PASSWORD           = dbutils.secrets.get(scope="randy_pitcher_workspace", key="SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT_IDENTIFIER = dbutils.secrets.get(scope="randy_pitcher_workspace", key="SNOWFLAKE_ACCOUNT_IDENTIFIER")
SNOWFLAKE_DATABASE           = 'databricks_unity_catalog' # this should already exist in snowflake


catalog_properties = {
    'type':      'rest',
    'uri':       UC_CATALOG_URI,
    'token':     UC_CREDENTIAL,
    'warehouse': UC_CATALOG_TO_MIRROR
}
catalog = load_catalog(**catalog_properties)

In [0]:
# get tables to build mirrors for 💪
tables_to_mirror = []
namespaces_to_register = [namespace[0] for namespace in catalog.list_namespaces() if namespace[0] not in ['information_schema']]
for namespace in namespaces_to_register: 
  print(f'\n\nchecking {UC_CATALOG_TO_MIRROR}.{namespace}:')
  for _, tablename in catalog.list_tables(namespace):
    tables_to_mirror.append(catalog.load_table(f"{namespace}.{tablename}"))
    print(f"\tFound iceberg table: '{UC_CATALOG_TO_MIRROR}.{namespace}.{tables_to_mirror[-1].identifier[-1]}'")

In [0]:
import snowflake.connector

# Snowflake connection parameters
# These variables are gathered in the initial python cell above ⬆️
snowflake_config = {
    "user": SNOWFLAKE_USERNAME,
    "password": SNOWFLAKE_PASSWORD,
    "account": SNOWFLAKE_ACCOUNT_IDENTIFIER
}

# Create a connection object
snowflake_conn = snowflake.connector.connect(**snowflake_config)

In [0]:
# Validate the connection details before mirroring
try:
  curs = snowflake_conn.cursor()

  # set session configs
  curs.execute(f'use database {SNOWFLAKE_DATABASE}')

  # validation query
  curs.execute('select current_user(), current_role(), current_database(), current_warehouse(), 1=1 as warehouse_is_usable')
  row = curs.fetchone()
  print(f"""
    Snowflake connection validity check:
      - Current User:         '{row[0]}'
      - Current Role:         '{row[1]}'
      - Current Database:     '{row[2]}'
      - Current Warehouse:    '{row[3]}'
      - Warehouse is usable?: '{row[4]}'""")
  
except Exception as e:
  print(f'Snowflake connection error:\n{e}')

In [0]:
# Build snowflake mirrors 💪
# test query to validate the basics
curs = snowflake_conn.cursor()
for table_to_mirror in tables_to_mirror:
  try:    
    # grab the table name to use.
    # Note: If you expect naming collisions, you can prefix these by their schema name or whatever you like
    snowflake_mirror_schema = table_to_mirror.identifier[-2] # second to last part = schema
    snowflake_mirror_tablename = table_to_mirror.identifier[-1] # last part of the identifier is the table name
    metadata_file_path = '/'.join(table_to_mirror.metadata_location.split('/')[-3:])

    # create destination schema if it doesn't exist
    curs.execute(f'create schema if not exists {SNOWFLAKE_DATABASE}.{snowflake_mirror_schema}')

    # mirror the table
    mirror_query = f"""
      create or replace iceberg table {SNOWFLAKE_DATABASE}.{snowflake_mirror_schema}.{snowflake_mirror_tablename}
        external_volume = '{SNOWFLAKE_VOLUME}'
        catalog = '{SNOWFLAKE_CATALOG}'
        metadata_file_path = '{metadata_file_path}'
        copy grants;"""
    print(f"Attempting mirror command:{mirror_query}")
    curs.execute(mirror_query)
    print(f"✅ Success!\n\n")
    
  except Exception as e:
    print(f'❌ Failure. Snowflake mirroring error for table "{table_to_mirror.identifier[1]}.{table_to_mirror.identifier[2]}":\n{e}')
    